In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
from distutils.dir_util import copy_tree
fromDirectory='/content/drive/MyDrive/Captured frames/1000/fake'
toDirectory='/content/drive/MyDrive/Captured frames/train(700)/fake'
copy_tree(fromDirectory,toDirectory)

['/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_23.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_24.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_25.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_26.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_27.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_28.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_29.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_30.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/234_187_31.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/229_247_0.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/229_247_1.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/229_247_2.png',
 '/content/drive/MyDrive/Captured frames/train(700)/fake/229_247_3.png',
 '/content/drive/MyDrive/Captured frames/t

In [ ]:
input_shape = (224, 224, 3)

base_path = '/content/drive/MyDrive/Captured frames/'

image_gen = ImageDataGenerator(rescale=1./255.)
train_flow = image_gen.flow_from_directory(
    base_path + 'train(700)/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

image_gen1 = ImageDataGenerator(rescale=1./255.)
valid_flow = image_gen1.flow_from_directory(
    base_path + '200/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

'''

data_dir = '/content/drive/MyDrive/Captured frames/300'

real_data = [f for f in os.listdir(data_dir+'/real') if f.endswith('.png')]
fake_data = [f for f in os.listdir(data_dir+'/fake') if f.endswith('.png')]

X = []
Y = []

for img in real_data:
    X.append(img_to_array(load_img(data_dir+'/real/'+img)).flatten() / 255.0)
    Y.append(1)
for img in fake_data:
    X.append(img_to_array(load_img(data_dir+'/fake/'+img)).flatten() / 255.0)
    Y.append(0)

Y_val_org = Y

#Normalization
X = np.array(X)
Y = to_categorical(Y, 2)

#Reshape
X = X.reshape(-1, 224, 224, 3)

#Train-Test split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
'''


Found 6998 images belonging to 2 classes.
Found 3569 images belonging to 2 classes.


"\n\ndata_dir = '/content/drive/MyDrive/Captured frames/300'\n\nreal_data = [f for f in os.listdir(data_dir+'/real') if f.endswith('.png')]\nfake_data = [f for f in os.listdir(data_dir+'/fake') if f.endswith('.png')]\n\nX = []\nY = []\n\nfor img in real_data:\n    X.append(img_to_array(load_img(data_dir+'/real/'+img)).flatten() / 255.0)\n    Y.append(1)\nfor img in fake_data:\n    X.append(img_to_array(load_img(data_dir+'/fake/'+img)).flatten() / 255.0)\n    Y.append(0)\n\nY_val_org = Y\n\n#Normalization\nX = np.array(X)\nY = to_categorical(Y, 2)\n\n#Reshape\nX = X.reshape(-1, 224, 224, 3)\n\n#Train-Test split\nX_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)\n"

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0

from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


googleNet_model = EfficientNetB0(include_top=False, weights=None, input_shape=input_shape)
googleNet_model.trainable = True
model = Sequential()

model.add(googleNet_model)
model.add(GlobalAveragePooling2D())
#model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))
#model.add(Flatten())

#model.add(GlobalAveragePooling2D())
#model.add(Dense(units=2, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

'''
#Block 1
model.add(Conv2D(filters = 20, kernel_size=(3,3), strides=(3,3), 
                      padding="valid", activation='tanh', input_shape=(20, 100, 100))) #smooth stride; padding: handle edges;
model.add(MaxPooling2D(pool_size=(1,2)))

#Block 2
model.add(Conv2D(filters = 16, kernel_size=2, strides=1, 
                      padding="valid", activation='relu')) #smooth stride; padding: handle edges;
#model.add(tfkl.MaxPool2D(pool_size=2))
#model.add(tfkl.Dropout(.2))

#Block 3
model.add(Conv2D(filters = 8, kernel_size=2, strides=1, 
                      padding="valid", activation='relu')) #smooth stride; padding: handle edges;
#model.add(tfkl.MaxPool2D(pool_size=2))

#Flatten
model.add(GlobalMaxPooling2D())
model.add(Dense(1, activation='sigmoid'))

model.summary()
'''

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


'\n#Block 1\nmodel.add(Conv2D(filters = 20, kernel_size=(3,3), strides=(3,3), \n                      padding="valid", activation=\'tanh\', input_shape=(20, 100, 100))) #smooth stride; padding: handle edges;\nmodel.add(MaxPooling2D(pool_size=(1,2)))\n\n#Block 2\nmodel.add(Conv2D(filters = 16, kernel_size=2, strides=1, \n                      padding="valid", activation=\'relu\')) #smooth stride; padding: handle edges;\n#model.add(tfkl.MaxPool2D(pool_size=2))\n#model.add(tfkl.Dropout(.2))\n\n#Block 3\nmodel.add(Conv2D(filters = 8, kernel_size=2, strides=1, \n                      padding="valid", activation=\'relu\')) #smooth stride; padding: handle edges;\n#model.add(tfkl.MaxPool2D(pool_size=2))\n\n#Flatten\nmodel.add(GlobalMaxPooling2D())\nmodel.add(Dense(1, activation=\'sigmoid\'))\n\nmodel.summary()\n'

In [ ]:
#Currently not used
early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0,
                               patience=2,
                               verbose=0, mode='auto')
EPOCHS = 15
BATCH_SIZE = 100
history = model.fit(train_flow, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = valid_flow, verbose = 1)

Epoch 1/15
110/110 [==============================] - 92s 756ms/step - loss: 0.6890 - accuracy: 0.5579 - val_loss: 0.6946 - val_accuracy: 0.5038
Epoch 2/15
110/110 [==============================] - 78s 705ms/step - loss: 0.6789 - accuracy: 0.5875 - val_loss: 0.7007 - val_accuracy: 0.5038
Epoch 3/15
110/110 [==============================] - 78s 710ms/step - loss: 0.6704 - accuracy: 0.5992 - val_loss: 0.7089 - val_accuracy: 0.5038
Epoch 4/15
110/110 [==============================] - 79s 716ms/step - loss: 0.6663 - accuracy: 0.6089 - val_loss: 0.7224 - val_accuracy: 0.5038
Epoch 5/15
110/110 [==============================] - 79s 715ms/step - loss: 0.6592 - accuracy: 0.6167 - val_loss: 0.7328 - val_accuracy: 0.5038
Epoch 6/15
110/110 [==============================] - 79s 717ms/step - loss: 0.6554 - accuracy: 0.6233 - val_loss: 0.7330 - val_accuracy: 0.5038
Epoch 7/15
110/110 [==============================] - 80s 724ms/step - loss: 0.6520 - accuracy: 0.6230 - val_loss: 0.7295 - val_ac

KeyboardInterrupt: ignored

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))
t = f.suptitle('Pre-trained InceptionResNetV2 Transfer Learn with Fine-Tuning & Image Augmentation Performance ', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1,EPOCHS+1))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, EPOCHS+1, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch #')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, EPOCHS+1, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch #')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
#Output confusion matrix
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
    plt.ylabel('Actual label', size = 20)
    plt.xlabel('Predicted label', size = 20)
    plt.xticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.ylim([2, 0])
    plt.show()
    
print_confusion_matrix(Y_val_org, model.predict_classes(X))

In [ ]:
model.save('/content/drive/MyDrive/test/deepfake-detection-tensor.h5')